In [23]:
import numpy as np
from sklearn import linear_model
from sklearn.metrics import roc_auc_score
from sklearn.svm import SVC
import GO_utils
import utils

## Specify File Paths

In [2]:
gene2go_file_path = '../data/gene2go.txt' # If file doesn't exist, then run gene2go = download_ncbi_associations()
rpkm_file_path = '../../CS341_Data/transcript_rpkm_in_go_nonzero_exp.txt'
gene_count_file_path = '../data/supp_GO_term_gene_counts.txt'
biomart_file_path = '../data/biomart_ensembl_to_entrez.txt'  
sample_tissue_path = '../data/sampleID_tissue.txt'
obo_file_path = '../data/go-basic.obo'

## Get Gene Annotations for all GO terms in Supplementary File (include the GO descendant terms)

In [3]:
GO_terms = GO_utils.get_go_terms_descendants(biomart_file_path, gene2go_file_path, gene_count_file_path, obo_file_path, ev_codes=None)
GO_terms = GO_utils.sort_go_terms(GO_terms)
print 'Top GO terms'
for t in GO_terms[0:10]:
    print t.id, ' ', len(t.genes)

term = GO_terms[300]
print len(term.genes)
ensembl_ids = term.genes
ens_ids_dict = {}
for id in ensembl_ids:
    ens_ids_dict[id] = True

16439 GO terms associated with human NCBI Entrez GeneIDs


load obo file ../data/go-basic.obo


../data/go-basic.obo: format-version(1.2) data-version(releases/2016-04-27)


46518 nodes imported


Top GO terms
GO:0007166   2018
GO:0007186   1128
GO:0051960   741
GO:0050767   659
GO:0007167   652
GO:0045664   549
GO:0007169   483
GO:0019221   449
GO:0051962   432
GO:0002694   425
50


## 1st Pass Through Dataset: Obtain positive training examples

In [4]:
NUM_FEATURES = 8555
gene_features, positive_example_rows, gene_ids_ordered, num_transcripts = \
        GO_utils.get_positive_examples(rpkm_file_path, ens_ids_dict, NUM_FEATURES)

print 'After pass 1 (inserting positive examples), gene feature matrix has dimension: ', gene_features.shape
num_positive_examples = len(positive_example_rows)
num_negative_examples = num_positive_examples
num_examples = num_positive_examples + num_negative_examples
print 'num pos examples: ', num_positive_examples

After pass 1 (inserting positive examples), gene feature matrix has dimension:  (50, 8555)
num pos examples:  50


## 2nd Pass through dataset: Obtain an equal number of negative training exmaples

In [5]:
neg_rows = utils.rand_sample_exclude(range(0, num_transcripts), num_negative_examples, exclude=positive_example_rows)

gene_features_neg, gene_ids_ordered_neg = \
    GO_utils.get_negative_examples(rpkm_file_path, neg_rows, NUM_FEATURES)
gene_features = np.append(gene_features, gene_features_neg, axis=0)
gene_ids_ordered += gene_ids_ordered_neg

print 'After pass 2 (inserting negative examples), gene feature matrix has dimension: ', gene_features.shape

After pass 2 (inserting negative examples), gene feature matrix has dimension:  (100, 8555)


## Add Binary Labels to the Data and split into train/test

In [6]:
# Vector of labels for each example
labels = num_positive_examples * [1] + num_negative_examples * [0]

train, test = utils.split_data(gene_features, labels, gene_ids_ordered, train_set_size=0.7)
print 'Obtained training & testing data'

num examples:  100
Dimensionality of training set:  (70, 8555)
Dimensionality of test set:  (30, 8555)
Obtained training & testing data


## Evaluate Various Models

## Logistic Regression With 10-Fold CV, L2 Norm

In [60]:
num_folds = 10   # number of folds to use for cross-validation
loss_fxn = 'l2'  # Loss function to use. Must be either 'l1' or 'l2'
costs = np.logspace(-4, 4, 20)  # 10^(-arg1) to 10^arg2 in arg3 logarithmic steps
logreg_cv_L2 = linear_model.LogisticRegressionCV(Cs=costs, cv=num_folds, penalty=loss_fxn, tol=0.0001)
logreg_cv_L2.fit(train.gene_features, train.labels)
best_c = logreg_cv_L2.C_
pred_lr_cv_L2 = logreg_cv_L2.predict(test.gene_features)
utils.print_prediction_results('Cross-Validated Logistic Regression', test.labels, pred_lr_cv_L2,
                               other_info='Norm: ' + loss_function + ', # of Folds: ' + str(num_folds))
print 'Best cost (after inverting to obtain true cost): ', 1.0 / best_c

--------------------
Cross-Validated Logistic Regression
--------------------
Root Mean Square Error:  0.48304589154
ROC AUC Score:  0.772321428571
False positive rate:  0.166666666667
False negative rate:  0.0666666666667
Norm: l1, # of Folds: 10
Best cost (after inverting to obtain true cost):  [ 78.47599704]


## Logistic Regression With 3-Fold CV, L1 Norm

In [28]:
'''
num_folds = 10   # number of folds to use for cross-validation
loss_function = 'l1'  # Loss function to use. Must be either 'l1' or 'l2'
costs = np.logspace(-4, 4, 20)  # 10^(-start) to 10^stop in 10 logarithmic steps
logreg_cv_L1 = linear_model.LogisticRegressionCV(Cs=costs, cv=num_folds, penalty=loss_function, solver='liblinear', tol=0.0001)
logreg_cv_L1.fit(train.gene_features, train.labels)
best_c = logreg_cv_L1.C_
pred_lr_cv_L1 = logreg_cv_L1.predict(test.gene_features)
utils.print_prediction_results('Cross-Validated Logistic Regression', test.labels, pred_lr_cv_L1, 
                         other_info='Norm: ' + loss_function + ', # of Folds: ' + str(num_folds))
print 'Best cost (after inverting to obtain true cost): ', 1.0 / best_c

# Save results
out_fname = '../data/result_logreg_' + term.id + '.txt'
save_prediction_results(out_fname, term.id, 'Logistic Regresion with L1 Penalty', 
                              test, pred_lr_cv_L1, num_folds, 
                              tissue_set=None, best_cost=best_c)
                              '''

print_prediction_results('Cross-Validated Logistic Regression', test.labels, pred_lr_cv_L1, 
                         other_info='Norm: ' + loss_function + ', # of Folds: ' + str(num_folds))

--------------------
Cross-Validated Logistic Regression
--------------------
ROC AUC Score:  0.599547511312
False positive rate:  0.25
False negative rate:  0.5
Norm: l1, # of Folds: 10
done


## SVM

In [ ]:
from sklearn.grid_search import GridSearchCV

# Split training data into train and dev
#train_cv , dev = utils.split_data(train.gene_features, train.labels, train.gene_ids_ordered, train_set_size=0.7)

svc = SVC(kernel='rbf')
Cs = np.logspace(-6, -1, 10)
clf = GridSearchCV(estimator=svc, param_grid=dict(C=Cs),n_jobs=-1)    
clf.fit(train.gene_features, train.labels)
print 'Best score: ', clf.best_score_
best_C = clf.best_estimator_.C
print 'Best C: ', best_C

In [ ]:
clf = SVC(kernel='rbf', C=best_C)
clf.fit(train.gene_features, train.labels)
pred_svm = clf.predict(test.gene_features)
utils.print_prediction_results('SVM', test.labels, pred_svm)

In [ ]:
for t in GO_terms[0:100]:
    print t.id, ' ', len(t.genes)

In [27]:
from math import exp
costs = np.logspace(-4, 4, 10)
print costs

[  1.00000000e-04   7.74263683e-04   5.99484250e-03   4.64158883e-02
   3.59381366e-01   2.78255940e+00   2.15443469e+01   1.66810054e+02
   1.29154967e+03   1.00000000e+04]


In [79]:
utils.printhi()

AttributeError: 'module' object has no attribute 'printhi'

In [20]:

def save_prediction_results(fname, GO_id, model, test_data, preds, n_folds=None,
                            tissue_set=None, best_cost=None, other_info=None):
    """
    Save results of prediction for a given GO term to a text file.

    :param fname: Name of output file
    :param GO_id: ID of the GO term
    :param model: Type of model used (e.g. "Logistic Regression with L1 Norm")
    :param test_data: GeneData object containing test data
    :param preds: Binary predictions for test data
    :param n_folds: # of folds used for cross-validation
    :param tissue_set: Set of tissues used. If no option is specified, it's assumed that all tissues were used
    :param best_cost: If applicable, the best cost parameter (as determined by cross-validation)
    :param other_info: Other info to store in header of output file
    :return: None
    """

    out_file = open(fname, 'w')
    out_file.write('# Prediction results for GO term: ' + GO_id + '\n')
    out_file.write('# Model used: ' + model + '\n')
    auc_score = roc_auc_score(test_data.labels, preds)

    print 'auc: ', auc_score

    out_file.write('# ROC AUC Score: ' + str(auc_score) + '\n')
    if tissue_set:
        out_file.write('# Tissues used: ' + tissue_set + '\n')
    else:
        out_file.write('# All tissues were included\n')
    if n_folds:
        out_file.write('# Number of folds used for cross-validation: ' + str(n_folds) + '\n')
    if best_cost:
        out_file.write('# Best cost parameter (determined by CV): ' + str(best_cost) + '\n')
    if other_info:
        out_file.write('# ' + other_info + '\n')

    # Writ out the predictions
    out_file.write('# Gene ID\tLabel\tPrediction\n')
    for id,label,pred in zip(test_data.gene_ids_ordered, test_data.labels, preds):
        out_file.write(id + '\t' + str(label) + '\t' + str(pred) + '\n')

    out_file.close()
